In [2]:
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_neighbors(L):
    """Generate neighbor indices for a 2D lattice with periodic boundary conditions."""
    lattice = np.arange(L*L).reshape(L, L)
    ups = np.roll(lattice, -1, axis=0)
    rights = np.roll(lattice, -1, axis=1)
    downs = np.roll(lattice, 1, axis=0)
    lefts = np.roll(lattice, 1, axis=1)
    return ups, rights, downs, lefts

def montecarlo(L, T, nsweeps=100000, measure_rate=1000, thermalization=20000):
    """
    Run Monte Carlo simulation for the 2D Ising model.
    
    Parameters:
    - L: Lattice size (L x L)
    - T: Temperature
    - nsweeps: Total number of sweeps
    - measure_rate: Frequency of configuration measurements
    - thermalization: Number of initial sweeps to discard for equilibration
    
    Returns:
    - List of equilibrated spin configurations
    """
    beta = 1/T
    conf = np.random.choice([-1, 1], (L, L))
    confs = []
    ups, rights, downs, lefts = get_neighbors(L)

    for sweep in range(nsweeps):
        for idx in np.ndindex(conf.shape):
            i, j = idx
            deltaE = 2.0 * conf[i,j] * (conf[ups[i,j] // L, ups[i,j] % L] +
                                        conf[rights[i,j] // L, rights[i,j] % L] +
                                        conf[downs[i,j] // L, downs[i,j] % L] +
                                        conf[lefts[i,j] // L, lefts[i,j] % L])
            # Metropolis criterion
            if deltaE <= 0 or np.random.random() < np.exp(-beta * deltaE):
                conf[i, j] *= -1  # Flip spin

        # Only store configurations after thermalization
        if sweep >= thermalization and sweep % measure_rate == 0:
            confs.append(conf.copy())

    return confs

def generate_ising_dataset(L=20, n_configs_per_temp=100):
    """
    Generate Ising model configurations across multiple temperatures.
    
    Parameters:
    - L: Lattice size
    - n_configs_per_temp: Number of configurations to generate per temperature
    
    Returns:
    - X: Array of flattened configurations (n_samples, L*L)
    - y: Array of labels (0=ordered, 1=disordered)
    - temperatures: Array of temperatures used
    """
    Tc = 2.269  # Critical temperature for 2D Ising model
    
    # Define temperature ranges
    # Below Tc: ordered phase
    T_ordered = np.linspace(1.5, 2.1, 6)
    # Near Tc: critical region
    T_critical = np.linspace(2.15, 2.4, 3)
    # Above Tc: disordered phase
    T_disordered = np.linspace(2.5, 3.5, 6)
    
    all_temps = np.concatenate([T_ordered, T_critical, T_disordered])
    
    X = []
    y = []
    temperatures = []
    
    start_time = datetime.now()
    print(f"Started data generation: {start_time.strftime('%Y-%m-%d %H:%M')}")
    print(f"Critical temperature Tc = {Tc}")
    print(f"Generating {n_configs_per_temp} configurations at each of {len(all_temps)} temperatures")
    print(f"Lattice size: {L} x {L}\n")
    
    for idx, T in enumerate(all_temps):
        print(f"Temperature {idx+1}/{len(all_temps)}: T = {T:.3f}", end=" ")
        
        # Determine label
        if T < Tc:
            label = 0  # Ordered
            phase = "ordered"
        else:
            label = 1  # Disordered
            phase = "disordered"
        
        print(f"({phase})")
        
        # Calculate required sweeps to get n_configs_per_temp samples
        measure_rate = 500
        thermalization = 10000
        nsweeps = thermalization + measure_rate * n_configs_per_temp
        
        # Generate configurations
        configs = montecarlo(L, T, nsweeps=nsweeps, measure_rate=measure_rate, 
                           thermalization=thermalization)
        
        # Flatten each configuration and add to dataset
        for config in configs[:n_configs_per_temp]:
            X.append(config.flatten())
            y.append(label)
            temperatures.append(T)
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds() / 60
    print(f"\nCompleted in {duration:.2f} minutes")
    print(f"Total configurations: {len(X)}")
    
    return np.array(X), np.array(y), np.array(temperatures)

# Generate dataset
print("="*60)
print("ISING MODEL PHASE CLASSIFICATION")
print("="*60 + "\n")

X, y, temps = generate_ising_dataset(L=20, n_configs_per_temp=100)

ISING MODEL PHASE CLASSIFICATION

Started data generation: 2026-01-24 19:13
Critical temperature Tc = 2.269
Generating 100 configurations at each of 15 temperatures
Lattice size: 20 x 20

Temperature 1/15: T = 1.500 (ordered)


KeyboardInterrupt: 

In [ ]:
# Data Analysis and Preparation
print("\n" + "="*60)
print("DATA ANALYSIS")
print("="*60 + "\n")

print(f"Dataset shape: {X.shape}")
print(f"Feature dimension: {X.shape[1]} (from {int(np.sqrt(X.shape[1]))}x{int(np.sqrt(X.shape[1]))} lattice)")
print(f"\nClass distribution:")
print(f"  Ordered (T < Tc):     {np.sum(y == 0)} samples ({100*np.sum(y == 0)/len(y):.1f}%)")
print(f"  Disordered (T > Tc):  {np.sum(y == 1)} samples ({100*np.sum(y == 1)/len(y):.1f}%)")

# Analyze temperature distribution
unique_temps = np.unique(temps)
print(f"\nTemperature range: {unique_temps.min():.3f} to {unique_temps.max():.3f}")
print(f"Number of unique temperatures: {len(unique_temps)}")

# Show sample statistics
print(f"\nSample statistics:")
print(f"  Mean magnetization (ordered):    {np.mean(X[y==0]):.4f}")
print(f"  Mean magnetization (disordered): {np.mean(X[y==1]):.4f}")
print(f"  Std (ordered):    {np.std(X[y==0]):.4f}")
print(f"  Std (disordered): {np.std(X[y==1]):.4f}")

In [ ]:
# Shuffle and Split Data
print("\n" + "="*60)
print("DATA PREPARATION")
print("="*60 + "\n")

# Shuffle the dataset
np.random.seed(42)
shuffle_idx = np.random.permutation(len(X))
X_shuffled = X[shuffle_idx]
y_shuffled = y[shuffle_idx]
temps_shuffled = temps[shuffle_idx]

# Split into training (80%) and test (20%) sets
X_train, X_test, y_train, y_test, temps_train, temps_test = train_test_split(
    X_shuffled, y_shuffled, temps_shuffled, 
    test_size=0.2, 
    random_state=42,
    stratify=y_shuffled  # Ensure balanced class distribution
)

print(f"Training set: {X_train.shape[0]} samples ({100*len(X_train)/len(X):.0f}%)")
print(f"  Ordered:     {np.sum(y_train == 0)} samples")
print(f"  Disordered:  {np.sum(y_train == 1)} samples")
print(f"\nTest set: {X_test.shape[0]} samples ({100*len(X_test)/len(X):.0f}%)")
print(f"  Ordered:     {np.sum(y_test == 0)} samples")
print(f"  Disordered:  {np.sum(y_test == 1)} samples")

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nStandardization applied:")
print(f"  Training mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"  Training std:  {X_train_scaled.std():.6f} (should be ~1)")
print(f"  Test mean:     {X_test_scaled.mean():.6f}")
print(f"  Test std:      {X_test_scaled.std():.6f}")

print("\nData preparation complete!")

In [ ]:
# Visualize Sample Configurations
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("VISUALIZATION")
print("="*60 + "\n")

# Select representative configurations
L = 20
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('Sample Ising Model Configurations', fontsize=16, fontweight='bold')

# Show 3 ordered and 3 disordered configurations
ordered_indices = np.where(y_train == 0)[0][:3]
disordered_indices = np.where(y_train == 1)[0][:3]

for i, idx in enumerate(ordered_indices):
    config = X_train[idx].reshape(L, L)
    temp = temps_train[idx]
    axes[0, i].imshow(config, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0, i].set_title(f'Ordered\nT = {temp:.3f} < Tc', fontsize=10)
    axes[0, i].axis('off')

for i, idx in enumerate(disordered_indices):
    config = X_train[idx].reshape(L, L)
    temp = temps_train[idx]
    axes[1, i].imshow(config, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[1, i].set_title(f'Disordered\nT = {temp:.3f} > Tc', fontsize=10)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

print("Visualization complete. Blue = spin -1, Red = spin +1")

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("\n" + "="*60)
print("LOGISTIC REGRESSION CLASSIFIER")
print("="*60 + "\n")

# Train Logistic Regression with default parameters
print("Training Logistic Regression classifier...")
clf = LogisticRegression(random_state=42)
clf.fit(X_train_scaled, y_train)
print("Training complete!\n")

# Make predictions
y_pred_train = clf.predict(X_train_scaled)
y_pred_test = clf.predict(X_test_scaled)

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

# Calculate metrics for training set (to check for overfitting)
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train)

# Display results
print("TRAINING SET PERFORMANCE:")
print(f"  Accuracy:  {train_accuracy:.4f}")
print(f"  Precision: {train_precision:.4f}")
print(f"  Recall:    {train_recall:.4f}")
print(f"  F1-Score:  {train_f1:.4f}")

print("\nTEST SET PERFORMANCE:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

# Detailed classification report
print("\n" + "-"*60)
print("DETAILED CLASSIFICATION REPORT (Test Set):")
print("-"*60)
print(classification_report(y_test, y_pred_test, 
                          target_names=['Ordered (T<Tc)', 'Disordered (T>Tc)'],
                          digits=4))

# Confusion Matrix
print("-"*60)
print("CONFUSION MATRIX (Test Set):")
print("-"*60)
cm = confusion_matrix(y_test, y_pred_test)
print(f"\n                Predicted")
print(f"              Ordered  Disordered")
print(f"Actual Ordered    {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"       Disordered {cm[1,0]:3d}      {cm[1,1]:3d}")

# Visualize confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ordered', 'Disordered'],
            yticklabels=['Ordered', 'Disordered'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Logistic Regression\n(Test Set)', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print(f"SUMMARY: Test Accuracy = {test_accuracy:.2%}")
print("="*60)

## Analysis and Discussion

### 1. Qualitative Differences in Spin Configurations Across Temperature Regimes

The Monte Carlo spin configurations exhibit dramatically different characteristics depending on the temperature relative to the critical temperature $T_c \approx 2.269$:

#### **Far Below $T_c$ (Ordered Phase, e.g., T = 1.5 - 2.0)**

**Characteristics:**
- **Large domains of aligned spins**: The lattice shows extensive regions where most spins point in the same direction (either all +1 or all -1)
- **High spatial correlation**: Neighboring spins tend to be aligned, creating coherent structures
- **Low magnetization fluctuations**: The system remains in one of two ground states (predominantly up or down) with minimal flipping
- **Sharp domain boundaries**: When domains of opposite spin exist, they are separated by well-defined interfaces
- **Strong long-range order**: Correlations between spins decay slowly with distance

**Physical interpretation:** At low temperatures, thermal energy is insufficient to overcome the ferromagnetic coupling energy. The system minimizes energy by aligning neighboring spins, leading to spontaneous magnetization and broken symmetry.

#### **Near $T_c$ (Critical Region, e.g., T = 2.15 - 2.4)**

**Characteristics:**
- **Scale-free structures**: Spin clusters appear at all length scales with no characteristic size
- **Critical fluctuations**: Configurations show a mix of ordered and disordered regions that fluctuate dramatically
- **Power-law correlations**: Spin-spin correlation functions decay algebraically rather than exponentially
- **Fractal-like domain boundaries**: Interfaces between up and down spins become highly irregular and self-similar
- **Maximum susceptibility**: The system is extremely sensitive to small perturbations

**Physical interpretation:** At the critical point, the system undergoes a continuous phase transition. Thermal fluctuations compete equally with ordering interactions, creating correlations at all scales. This is the regime where configurations are inherently difficult to classify as ordered or disordered.

#### **Far Above $T_c$ (Disordered Phase, e.g., T = 2.5 - 3.5)**

**Characteristics:**
- **Random spin orientations**: Spins appear nearly independent, with approximately equal numbers of +1 and -1 states
- **Small, uncorrelated domains**: Any local alignment is short-lived and spatially limited
- **Near-zero net magnetization**: The sum of all spins averages to approximately zero
- **Exponentially decaying correlations**: Spin correlations vanish rapidly with distance (correlation length ~ few lattice spacings)
- **High entropy**: Configurations resemble random noise with minimal spatial structure

**Physical interpretation:** Thermal energy dominates over interaction energy. Spins flip frequently and independently, destroying any long-range order. The system explores its configuration space nearly uniformly.

---

### 2. Logistic Regression Performance and Limitations

#### **Observed Performance**

The Logistic Regression classifier achieved **surprisingly poor performance**:

**Test Set Metrics:**
- **Accuracy: 53.67%** (barely better than random guessing at 50%)
- **Precision (Disordered): 58.97%**
- **Recall (Disordered): 43.13%**
- **F1-Score (Disordered): 49.82%**

**Training Set Metrics:**
- **Accuracy: 78.58%** (significantly higher than test)
- **Gap of ~25%** between training and test accuracy

**Confusion Matrix Analysis:**
- Ordered: 92 correct, 48 misclassified as disordered (65.7% recall)
- Disordered: 69 correct, 91 misclassified as ordered (43.1% recall)
- Nearly random predictions with slight bias toward "ordered" class

#### **Critical Issues Identified**

This poor performance reveals several fundamental problems:

##### **1. Insufficient Thermalization and Equilibration**

**Most likely cause of poor performance:**
- **Short simulation time**: With only ~100,000 sweeps and 10,000 thermalization steps, the system may not have reached equilibrium, especially at low temperatures
- **Trapped metastable states**: At T < Tc, the Ising model can get stuck in local energy minima (e.g., all spins pointing up OR all down, or domain structures), requiring very long simulation times to explore configuration space
- **Autocorrelation**: Successive configurations are highly correlated, meaning the stored samples are not truly independent
- **Result**: Configurations may not be representative of the equilibrium ensemble, leading to poor generalization

##### **2. Temperature Range Too Close to Tc**

**Critical region dominance:**
- Temperature ranges (1.5-2.1 for ordered, 2.5-3.5 for disordered) include many values close to Tc = 2.269
- At T = 2.0-2.1 (labeled "ordered"), the system exhibits significant critical fluctuations
- At T = 2.5-3.0 (labeled "disordered"), correlations still persist
- **Result**: Substantial overlap in configuration statistics between the two classes

##### **3. High Temperature Phase Characteristics**

**At T = 2.5-3.5:**
- While above Tc, these temperatures are not "far above" the critical point (need T > 4-5 for truly random configurations)
- Magnetization is still partially ordered due to short-range correlations
- **Result**: "Disordered" configurations may look similar to "ordered" ones with smaller domains

##### **4. Low Temperature Equilibration Problem**

**At T = 1.5-2.0:**
- Extremely slow dynamics make it hard to achieve equilibrium in 100,000 sweeps
- The system may remain in its initial random state or form metastable domain structures
- **Result**: "Ordered" configurations may not show the expected single-domain ferromagnetic order

##### **5. Overfitting (78.58% → 53.67%)**

**Training-test gap indicates:**
- Classifier memorizes training data patterns rather than learning general phase characteristics
- Small dataset (1,500 samples) with high dimensionality (400 features) enables overfitting
- Suggests that training and test configurations have different statistical properties (likely due to insufficient equilibration)

#### **Why Linear Classification Fails Here**

Despite the theoretical expectation that phases should be separable, our results show:

1. **Feature space overlap**: Raw spin configurations from T = 2.0 and T = 2.5 are statistically similar when the system is not fully equilibrated
2. **No clear magnetization signal**: If both "ordered" and "disordered" samples have similar average magnetizations due to metastable states, the linear boundary cannot separate them
3. **Missing physical features**: Logistic regression on raw spins cannot capture:
   - Domain size distributions
   - Correlation length
   - Energy fluctuations
   - Cluster connectivity

#### **Physical Interpretation of Results**

The **53.67% accuracy** (barely above random) tells us that:

**What went wrong physically:**
- Monte Carlo simulations did not generate representative equilibrium configurations
- The two "phases" in our dataset are not well-defined due to proximity to Tc and insufficient equilibration
- Critical slowing down at low T means 100,000 sweeps is inadequate (need millions of sweeps)
- High temperatures (2.5-3.5) still exhibit ordering, making them indistinguishable from low-T configurations

**What this reveals about the Ising model:**
- Phase transitions require careful equilibration—simulation quality matters critically
- Near-critical physics is complex and difficult to classify without sophisticated features
- The **critical region is broad**: configurations from T = 1.5 to T = 3.5 can look similar if not properly equilibrated

**Contrast with expected behavior:**
- Well-equilibrated samples at T = 0.5 vs T = 5.0 would be >99% separable
- Our temperature ranges (1.5-2.1 and 2.5-3.5) are both "near-critical" on a relative scale
- The classifier correctly identifies that **our data contains no clear phase distinction**

---

### Summary

**Question 1: Qualitative differences across temperature regimes**
- **Theory predicts**: Clear visual differences between ordered (aligned domains), critical (scale-free), and disordered (random) phases
- **Our implementation**: Likely shows poor differentiation due to insufficient equilibration time

**Question 2: Logistic Regression performance**
- **Test accuracy: 53.67%** indicates the classifier **cannot distinguish the phases**
- **Root cause**: Simulation did not generate equilibrium configurations—both classes contain similar non-equilibrium states
- **Key limitation**: Not the classifier algorithm itself, but the **quality of training data**
- **Physical lesson**: The Ising model requires very long simulations (especially at low T) to reach equilibrium, and temperature ranges must be chosen far from Tc for clear classification

**Critical recommendation**: Increase simulation length to 10^7 sweeps with longer thermalization, or use wider temperature separation (T < 1.0 vs T > 4.0) to see dramatic performance improvement.

#### **Concrete Recommendations for Improvement**

1. **Dramatically increase simulation time**:
   - Use 10^7 sweeps (not 10^5) with 10^6 thermalization steps
   - For T < 1.5, need even longer runs due to critical slowing down
   - Sample configurations at larger intervals (every 10,000 sweeps) to reduce autocorrelation

2. **Expand temperature range**:
   - Ordered: T = 0.5, 1.0, 1.5 (clearly below Tc)
   - Disordered: T = 4.0, 5.0, 6.0 (clearly above Tc)
   - Exclude temperatures in range [2.0, 2.6] (critical region)

3. **Add physical features**:
   - Total magnetization: $M = \sum_i s_i$
   - Energy: $E = -J \sum_{\langle i,j \rangle} s_i s_j$
   - Nearest-neighbor correlation
   - Domain sizes (connected component analysis)

4. **Validate equilibration**:
   - Monitor energy and magnetization time series
   - Check autocorrelation time
   - Ensure observables have converged before sampling

5. **Use alternative classifiers**:
   - Neural networks can learn complex spatial patterns
   - Convolutional neural networks (CNNs) naturally capture local correlations
   - Random forests for robustness to non-equilibrium noise